In [163]:
import pandas as pd
import datetime
import seaborn as sn
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [146]:
df = pd.read_csv('../Data/data_original.csv')


In [147]:
df.info()

## drop trade ##

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41256 entries, 0 to 41255
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   unix         41256 non-null  int64  
 1   date         41256 non-null  object 
 2   symbol       41256 non-null  object 
 3   open         41256 non-null  float64
 4   high         41256 non-null  float64
 5   low          41256 non-null  float64
 6   close        41256 non-null  float64
 7   Volume BTC   41256 non-null  float64
 8   Volume USDT  41256 non-null  float64
 9   tradecount   15320 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 3.1+ MB


In [148]:
df.drop("tradecount", axis = 1, inplace = True)

## Date formatting
not all dates are in 24 hour format, 
some are in AM/PM format with unix granularity in seconds and some with granularity in milliseconds

In [150]:
df['date_12_hour'] = df['date'].astype(str).str.contains("M")

In [151]:
df['date_12_hour'].value_counts(normalize = True).mul(100).round(1).astype(str) + '%'

True     62.9%
False    37.1%
Name: date_12_hour, dtype: object

In [152]:
df['unix_digits'] = df['unix'].astype(str).str.len()

In [153]:
df['unix_digits'].value_counts(normalize = True).mul(100).round(1).astype(str) + '%'

10    62.9%
13    37.1%
Name: unix_digits, dtype: object

In [154]:
pd.crosstab(df['date_12_hour'], df['unix_digits'])

unix_digits,10,13
date_12_hour,,
False,0,15320
True,25936,0


In [155]:
def f(row):
    if row['unix'] > 1e+12:
        val = pd.to_datetime(row['unix'], unit='ms', origin='unix')
    else:
        val = pd.to_datetime(row['unix'], unit='s', origin='unix')
    return val

In [156]:
df['date2'] = df.apply(f, axis = 1)

In [157]:
df.tail()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,date_12_hour,unix_digits,date2
41251,1502956800,2017-08-17 08-AM,BTC/USDT,4349.99,4377.85,4333.32,4360.69,0.9499,4139.70,True,10,2017-08-17 08:00:00
41252,1502953200,2017-08-17 07-AM,BTC/USDT,4324.35,4349.99,4287.41,4349.99,4.4400,19241.06,True,10,2017-08-17 07:00:00
41253,1502949600,2017-08-17 06-AM,BTC/USDT,4315.32,4345.45,4309.37,4324.35,7.2300,31282.31,True,10,2017-08-17 06:00:00
41254,1502946000,2017-08-17 05-AM,BTC/USDT,4308.83,4328.69,4291.37,4315.32,23.2300,100304.82,True,10,2017-08-17 05:00:00
41255,1502942400,2017-08-17 04-AM,BTC/USDT,16199.91,16199.91,4261.32,4308.83,44.5100,190952.85,True,10,2017-08-17 04:00:00


## Exploring Price

In [165]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['date2'], y=df['close'], mode='lines', name='close price'))


In [179]:
freq = 'M'
freq_df = df.groupby(pd.Grouper(key='date2', axis=0, freq=freq)).agg({'open':'first','close':'last','high':'max', 'low':'min'})

In [180]:
freq_df.reset_index(inplace=True)

In [181]:
freq_df.head()

,date2,open,close,high,low
0,2017-08-31,16199.91,4724.89,16199.91,3400.00
1,2017-09-30,4724.89,4378.51,4939.19,2817.00
2,2017-10-31,4378.51,6463.00,6498.01,4110.00
3,2017-11-30,6463.00,9838.96,11300.03,5325.01
4,2017-12-31,9838.96,13716.36,19798.68,9380.00


In [186]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=freq_df['date2'], y=freq_df['close'], mode='lines', name='close price'))
fig.add_trace(go.Scatter(x=freq_df['date2'], y=freq_df['high'], mode='lines', name='high price'))
fig.add_trace(go.Scatter(x=freq_df['date2'], y=freq_df['low'], mode='lines', name='low price'))
fig.add_trace(go.Scatter(x=freq_df['date2'], y=freq_df['open'], mode='lines', name='open price'))
# fig.show()
